In [1]:
import sys
import os
# 1. Get the current directory of the notebook (e.g. .../ai-research-assistant/notebooks)
# 2. Go up one level to the project root (..)
# 3. Add that root to Python's search list (sys.path)
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
print(sys.executable)

d:\Learning\personal-research-assistant-with-multi-source-rag\ai-research-assistant\.venv\Scripts\python.exe


In [3]:
from src.utils.config import Config
from src.document_loader import DocumentLoader, Document
from src.vector_store import VectorStore
from groq import Groq

print("✅ Imports successful!")

LLM_PROVIDER: groq
EMBEDDING_PROVIDER: local
EMBEDDING_MODEL: multi-qa-distilbert-cos-v1
VECTOR_SIZE: 768
USE_INTELLIGENT_CHUNKING: False
✅ Imports successful!


In [4]:
document = Document(
     "Python is a high-level programming language. It's great for beginners and experts alike.", 
    {"source_type": "test", "title": "Python Intro"}
)
print(vars(document))

{'content': "Python is a high-level programming language. It's great for beginners and experts alike.", 'metadata': {'source_type': 'test', 'title': 'Python Intro'}}


In [6]:
vector_store = VectorStore(in_memory=True)
vars(vector_store)

Initializing collection: research_documents
[OK] Created collection: research_documents


{'collection_name': 'research_documents',
 'qdrant_client': <qdrant_client.qdrant_client.QdrantClient at 0x22666364210>,
 'groq_client': <groq.Groq at 0x226663d9c50>,
 'openai_client': None,
 'local_model': SentenceTransformer(
   (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'DistilBertModel'})
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
   (2): Normalize()
 )}

In [7]:
groq_client = Groq(api_key=Config.GROQ_API_KEY)
print("✅ Components initialized!")

✅ Components initialized!


#### Test single-source RAG

In [8]:
test_content = """
Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval 
with text generation. It works by first retrieving relevant documents from a knowledge base, 
then using those documents as context for generating responses.

The main benefits of RAG include:
1. Access to up-to-date information
2. Reduced hallucinations
3. Ability to cite sources
4. Cost-effective compared to fine-tuning

RAG is particularly useful for question-answering systems, chatbots, and research assistants.
"""

doc = Document(
    content=test_content,
    metadata={"source_type": "test", "title": "RAG Introduction"}
)

In [10]:
vector_store.add_documents([doc])
print("✅ Document added to vector store!")

  0%|          | 0/1 [00:00<?, ?it/s]

   [AI Research Assistant] Using sliding window chunking for document: False
[OK] Added 1 chunks from 1 documents
✅ Document added to vector store!


#### The response without `stream`

In [ ]:
# Test query
question = input()

# 1. Retrieve relevant context
results = vector_store.search(question, top_k=2)
# Process the results from the vector store for AI to use as context
context = "\n\n".join([r["text"] for r in results])

# Just print the results for the vector store
print(f"\nQuery: {question}\n")
for i, result in enumerate(results, 1):
    print(f"Result {i} (score: {result['score']:.3f}):")
    print(f"{result['text']}\n")
    
print("*"*60)

# 2. Create prompt with context
messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Answer questions based on the provided context. If the context doesn't contain the answer, say so."
        },
        {
            "role": "user",
            "content": f"Context:\n{context}\n\nQuestion: {question}"
        }
    ]

# 3. Generate response
response = groq_client.chat.completions.create(
    model=Config.GROQ_MODEL,
    messages=messages,
    max_tokens=Config.MAX_TOKENS,
    temperature=Config.TEMPERATURE,
)

# 4. Print response without `Stream`
print(response.choices[0].message.content)


Query: What is RAG and it benefit?

Result 1 (score: 0.734):

Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval 
with text generation. It works by first retrieving relevant documents from a knowledge base, 
then using those documents as context for generating responses.

The main benefits of RAG include:
1. Access to up-to-date information
2. Reduced hallucinations
3. Ability to cite sources
4. Cost-effective compared to fine-tuning

RAG is particularly useful for question-answering systems, chatbots, and research assistants.


************************************************************
RAG (Retrieval-Augmented Generation) is a technique that combines information retrieval with text generation. It works by first retrieving relevant documents from a knowledge base, then using those documents as context for generating responses.

The benefits of RAG include:
1. Access to up-to-date information
2. Reduced hallucinations
3. Ability to cite sources
4.

#### The response with `stream`

In [ ]:
# Test query
question = input()

# 1. Retrieve relevant context
results = vector_store.search(question, top_k=2)
# Process the results from the vector store for AI to use as context
context = "\n\n".join([r["text"] for r in results])

# Just print the results for the vector store
print(f"\nQuery: {question}\n")
for i, result in enumerate(results, 1):
    print(f"Result {i} (score: {result['score']:.3f}):")
    print(f"{result['text']}\n")

print("*"*60)

# 2. Create prompt with context
messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Answer questions based on the provided context. If the context doesn't contain the answer, say so."
        },
        {
            "role": "user",
            "content": f"Context:\n{context}\n\nQuestion: {question}"
        }
    ]

# 3. Generate response
response = groq_client.chat.completions.create(
    model=Config.GROQ_MODEL,
    messages=messages,
    max_tokens=Config.MAX_TOKENS,
    temperature=Config.TEMPERATURE,
    stream=True,
)

# 4. Stream response
print("Answer: ", end="")
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)


Query: What is RAG and it benefit?

Result 1 (score: 0.734):

Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval 
with text generation. It works by first retrieving relevant documents from a knowledge base, 
then using those documents as context for generating responses.

The main benefits of RAG include:
1. Access to up-to-date information
2. Reduced hallucinations
3. Ability to cite sources
4. Cost-effective compared to fine-tuning

RAG is particularly useful for question-answering systems, chatbots, and research assistants.


************************************************************
Answer: RAG stands for Retrieval-Augmented Generation, which is a technique that combines information retrieval with text generation. The benefits of RAG include:

1. Access to up-to-date information
2. Reduced hallucinations
3. Ability to cite sources
4. Cost-effective compared to fine-tuning

This technique is particularly useful for applications such as questio

#### Test Multi-source RAG

In [40]:
project_root = Config.PROJECT_ROOT

# Initialize Document Loader
loader = DocumentLoader()
 
docs = []

# Load PDF
pdf_path = os.path.join(project_root, "data", "docker_cheatsheet.pdf")
#pdf_path = "D:/Learning/personal-research-assistant-with-multi-source-rag/ai-research-assistant/data/IBM SkillsBuild_AI Experiential Learning Lab_2025_Guide.pdf"
if os.path.exists(pdf_path):
    print(f"   Loading PDF: {pdf_path}")
    pdf_docs = loader.load_pdf(pdf_path)# [:3] Load first 3 pages for speed
    docs.extend(pdf_docs)

# Load Web
web_url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
print(f"   Loading Web: {web_url}")
docs.append(loader.load_web_page(web_url))
        
# Load YouTube
yt_url = "https://www.youtube.com/watch?v=aircAruvnKk"
print(f"   Loading YouTube: {yt_url}")
docs.append(loader.load_youtube_transcript(yt_url))

   Loading PDF: d:\Learning\personal-research-assistant-with-multi-source-rag\ai-research-assistant\data\docker_cheatsheet.pdf
   Loading Web: https://en.wikipedia.org/wiki/Artificial_intelligence
   Loading YouTube: https://www.youtube.com/watch?v=aircAruvnKk


In [41]:
vector_store.add_documents(docs)
print("✅ Document added to vector store!")

  0%|          | 0/3 [00:00<?, ?it/s]

   [AI Research Assistant] Using sliding window chunking for document: False
   [AI Research Assistant] Using sliding window chunking for document: False
   [AI Research Assistant] Using sliding window chunking for document: False
[OK] Added 397 chunks from 3 documents
✅ Document added to vector store!


In [52]:
# Test query
question = input("Enter your question: ")
results = vector_store.search(question, top_k=2)
# Process the results from the vector store for AI to use as context
context = "\n\n".join([r["text"] for r in results])

# Print the results to we get from the vector store 
# to verify to the semantic search base on the query
print(f"\nQuery: {question}\n")
#for i, result in enumerate(results, 1):
 #   print(f"Result {i} (score: {result['score']:.3f}):")
 #   print(f"{result['text']}\n")

print("=" * 60)

from IPython.display import display, Markdown

# 2. Create prompt with context
messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Answer questions based on the provided context. If the context doesn't contain the answer, say so."
        },
        {
            "role": "user",
            "content": f"Context:\n{context}\n\nQuestion: {question}"
        }
    ]

# 3. Generate response
response = groq_client.chat.completions.create(
    model=Config.GROQ_MODEL,
    messages=messages,
    max_tokens=Config.MAX_TOKENS,
    temperature=Config.TEMPERATURE,
    stream=True,
)

# 4. Stream response
print("Answer: ", end="")
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)


Query: What is AI and it benefit?

Answer: According to the context, Artificial Intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making.

The benefits of AI mentioned in the context include:

1. Providing real-time information on evacuation conditions.
2. Helping farmers increase yield and identify areas that need irrigation, fertilization, and pesticide treatments in agriculture.
3. Analyzing large amounts of data in astronomy to gain new scientific insights.
4. Assisting in various tasks such as predicting crop ripening time, monitoring soil moisture, operating agricultural robots, and detecting diseases and pests.
5. Enabling machines to perceive their environment and take actions to achieve defined goals.

Overall, AI has the potential to improve efficiency, productivity, and decision-making in various fields, including agriculture, astr

In [12]:
def rag_query(question: str) -> str:
    """Answer question using RAG"""
    # 1. Retrieve relevant context
    results = vector_store.search(question, top_k=3)
    context = "\n\n".join([r["text"] for r in results])
    
    # 2. Create prompt with context
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Answer questions based on the provided context. If the context doesn't contain the answer, say so."
        },
        {
            "role": "user",
            "content": f"Context:\n{context}\n\nQuestion: {question}"
        }
    ]
    
    # 3. Generate response
    response = groq_client.chat.completions.create(
        model=Config.GROQ_MODEL,
        messages=messages,
        max_tokens=Config.MAX_TOKENS,
        temperature=Config.TEMPERATURE
    )
    
    return response.choices[0].message.content

In [ ]:
"What is Artificial intelligence and what are its benefits?"

In [14]:
# Test it
question = input()
answer = rag_query(question)
print(f"{question}\n")
print(f"Answer: {answer}")

[OK] Found [ScoredPoint(id=86, version=0, score=0.6375214455269018, payload={'text': 'moisture, operate agricultural robots, conduct\npredictive analytics\n, classify livestock pig call emotions, automate greenhouses, detect diseases and pests, and save water.\nArtificial intelligence is used in astronomy to analyze increasing amounts of available data and applications, mainly for "classification, regression, clustering, forecasting, generation, discovery, and the development of new scientific insights." For example, it is used for discovering exoplanets, forecasting solar activity, and distinguishing between signals and instrumental effects in gravitational wave astronomy. Additionally, it could be used for activities in space, such as space exploration, including the analysis of data from space missions, real-time science decisions of spacecraft, space debris avoidance, and ', 'chunk_index': 71, 'total_chunks': 362, 'source_type': 'web', 'source_url': 'https://en.wikipedia.org/wiki/A

In [13]:
print("\n💰 Estimated cost for this notebook: ~$0.001")
print("Check your usage at: https://groq.com/usage")


💰 Estimated cost for this notebook: ~$0.001
Check your usage at: https://groq.com/usage
